# Recap : Utilize Pandas Dataframe and Requests 

### Use csv file with pandas and matplotlib module

In [ ]:
import pandas as pd # import pandas module and declare alias is pd

In [ ]:
import matplotlib.pyplot as plt # import matplotlib module and declare alias is plt

In [ ]:
import json
import requests

In [ ]:
%matplotlib inline 
# This line only need if you want to plot on jupyter notebook

### Declare global variable

In [ ]:
global filePath 
filePath = './SolarData.csv'

### Declare plot function

In [ ]:
def plotData(data): # Function Plot
    plt.xlabel('Time')
    plt.ylabel('Radiation')
    plt.title('Daily Solar Radiation')
    plt.plot(data['Radiation'],'r.-')
    #plt.gcf().autofmt_xdate()
    plt.show()

### Create Filter Function for Select Date of interest

In [ ]:
def SelOneDay(DateStr, df) : # This Function use to select data where DateStr = '9/29/2016'
    daily = df.loc[df['Data'].str.contains(DateStr)] # command for select only Data is contained DateStr
    return daily

### Load CSV File to pandas Dataframe 
Load CSV File with .read_csv method in pandas

In [ ]:
def LoadData(): # Function Load data to DataFrame
    df = pd.DataFrame() # Create Blank Dataframe
    df = pd.read_csv(filePath) # Read CSV File to pandas dataframe
    df.sort_values(by=['UNIXTime'])
    return df

In [ ]:
def main():
    print('Start Load Dataset')
    df = pd.DataFrame()
    df = LoadData()
    oneday = SelOneDay('9/27/2016', df) # You can Change Date String here
    plotData(oneday)
    return df

In [ ]:
if __name__ == '__main__':
    df = main()

In [ ]:
df.head(5)

### Now df variable that returned is a Data in CSV File that contains in Pandas DataFrame
##### you can access all data that you specific field name with df['fieldname'] 

In [ ]:
df['Humidity'].head(10) # Whole Data are queried but use head to limit it to show.

### Select single row and parse to Dict Variable example

In [ ]:
OneRow = df.iloc[0] # Use iloc for Select with index or label

In [ ]:
OneRow

In [ ]:
dataDict = OneRow.to_dict() # Use to_dict method for convert the Data Frame to Dict variable

In [ ]:
dataDict

In [ ]:
print(type(dataDict)) 

### Parse Dict to JSON Object

In [ ]:
DataJson = json.dumps(dataDict)

In [ ]:
print(type(DataJson))

In [ ]:
print(DataJson)

### This is Example for send REST with post method to Free RESTFul API Develop Host (HookBIN)
#### First you must go to https://hookbin.com and Click Button Create New End Point

![Page Logo](./images/1.png) 


##### Then you got the URL end Point you must use it as Backend endpoint to store your data
- Copy url and replace in url variable
- Use DataJson as a body
- waitting for the request got response 200 Status
#### When it show status code = 200 you must go back to hook.bin page and Refresh it to show your output.

![Page2 Logo](./images/2.png) 

In [ ]:
try:
        response = requests.post(
            url="https://hookb.in/K2JwGWp2",  # You must change URL EndPoint Here !!!!!
            headers={
                "Content-Type": "text/plain; charset=utf-8",
            },
            data= DataJson
        )
        print('Response HTTP Status Code: {status_code}'.format(status_code=response.status_code))
        print('Response HTTP Response Body: {content}'.format(content=response.content))

except requests.exceptions.RequestException:
        print('HTTP Request failed')

![Page3 Logo](./images/3.png) 

#### Side Note Idea : You can implement backend in area of url that you create and do any stuff for make python to something you want such as put the json data to database or use it as input for calculation somthing and return answer to frontend as HTTP Body again

## Example for get Weather Data with Latitude and Longitude 
### This Example use the OpenWeatherMap :  https://openweathermap.org/api 
### Let's Do it.
#### Note !! You can use google map to get Lat ,Lon 

In [ ]:
from IPython.display import IFrame
IFrame('https://www.google.com/maps/embed?pb=!1m18!1m12!1m3!1d31538.33703598354!2d100.04262477919507!3d13.804936192751265!2m3!1f0!2f0!3f0!3m2!1i1024!2i768!4f13.1!3m3!1m2!1s0x0%3A0x82f9a5fa3e0adedf!2sThe+Museum+Hotel!5e0!3m2!1sth!2sth!4v1521968397461', 1000, 450)

In [ ]:
Lat = '13.8049362'
Lon = '100.0426248' # Assigned Latitude and Longitude value here

### Try it Here

In [ ]:
try:
        response = requests.get(
            url="http://api.openweathermap.org/data/2.5/weather",
            params={
                "lat": Lat, 
                "lon": Lon,
                "appid": "e08026ccf45b9282e18d158606b39aef",
            },
            headers={
                "Content-Type": "application/octet-stream",
            },
        )
        print('Response HTTP Status Code: {status_code}'.format(status_code=response.status_code))
        #print('Response HTTP Response Body: {content}'.format(content=response.content))
        weather = response.content
except requests.exceptions.RequestException:
        print('HTTP Request failed')

In [ ]:
weatherJson = json.loads(weather) # Load Html Body to Dict Variable

In [ ]:
print(json.dumps(weatherJson, indent =2)) # Convert to readable JSON Format with indent Argument.

### How to get response value

In [ ]:
import datetime

In [ ]:
print("City name : "+weatherJson.get('name'))

print("Sky Description : "+weatherJson.get('weather')[0].get('description'))

print("Humidity : "+str(weatherJson.get('main')['humidity']))
print("Wind Speed : " + str(weatherJson.get('wind')['speed']))
print("Sunrise Time : "+ datetime.datetime.fromtimestamp(
        int(weatherJson.get('sys')['sunrise'])
    ).strftime('%Y-%m-%d %H:%M:%S'))

print("Sunset Time : "+datetime.datetime.fromtimestamp(
        int(weatherJson.get('sys')['sunset'])
     ).strftime('%Y-%m-%d %H:%M:%S'))

###  --------------------------------------------------------------------------------------------------
## How to 5 Day Every 3 Hours forecasting  from openweather api

In [ ]:
try:
        response = requests.get(
            url="http://api.openweathermap.org/data/2.5/forecast",
            params={
                "lat": Lat,
                "lon": Lon,
                "appid": "3847aaa8c64d824b67d3558bd3c40e4f",
            },
        )
        print('Response HTTP Status Code: {status_code}'.format(
            status_code=response.status_code))
        forecast = response.content
except requests.exceptions.RequestException:
        print('HTTP Request failed')

In [ ]:
forecastJson = json.loads(forecast)

In [ ]:
print(json.dumps(forecastJson, indent=2))

### In this result are contained 5 Days Every 3 Hours weather Forecast so we got 40 index of List

### This is a example to show datapoint in forecast result for 5 Days (get only Wind Speed)

In [ ]:
data = forecastJson.get('list')

In [ ]:
windSpeed = []
TimeStamp = []
Humidity = []

In [ ]:
for ind in range(len(data)):
    windSpeed.append(data[ind].get('wind')['speed'])
    TimeStamp.append(data[ind].get('dt_txt'))
    Humidity.append(data[ind].get('main')['humidity'])

In [ ]:
fig = plt.figure(figsize=(18,6), dpi=192)
plt.xticks(rotation=90)
plt.xlabel('TimeStamp')
plt.ylabel('WindSpeed (m/s)')
plt.title('5 Day Wind Speed Forecasting')
plt.plot_date(TimeStamp,windSpeed,'g.-')
# plt.gcf().autofmt_xdate()
plt.show()

In [ ]:
fig = plt.figure(figsize=(18,6), dpi=192)
plt.xticks(rotation=90)
plt.xlabel('TimeStamp')
plt.ylabel('Humidity')
plt.title('5 Day Humidity Forecasting')
plt.plot_date(TimeStamp,Humidity,'r.-')
# plt.gcf().autofmt_xdate()
plt.show()

## End